# Incremental learning on image classification

## Libraries and packages


In [1]:
!pip3 install 'torch==1.4.0'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'

     |████████████████████████████████| 753.4MB 22kB/s 
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
     |████████████████████████████████| 4.0MB 2.8MB/s 
  Found existing installation: torchvision 0.6.1+cu101
    Uninstalling torchvision-0.6.1+cu101:
      Successfully uninstalled torchvision-0.6.1+cu101
     |████████████████████████████████| 634kB 2.6MB/s 
  Created wheel for Pillow-SIMD: filename=Pillow_SIMD-7.0.0.post3-cp36-cp36m-linux_x86_64.whl size=1110316 sha256=d6b50447fc439940451f73bee09c8cf455eeaff7ef35bba3c33689e493805ae9
  Stored in directory: /root/.cache/pip/wheels/d3/ac/4f/4cdf8febba528e5f1b09fc58d5181e1c12ed1e8655dcd583b8
Successfully built Pillow-SIMD


In [2]:
import os
import urllib
import logging

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import Dataset, Subset, DataLoader, ConcatDataset
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import resnet34

from PIL import Image
from copy import deepcopy

import numpy as np

from sklearn.metrics import confusion_matrix

In [4]:
# Download packages from repository
!git clone https://github.com/manuelemacchia/incremental-learning-image-classification.git
!mv -v incremental-learning-image-classification/* .
!rm -rf incremental-learning-image-classification README.md

from data.cifar100 import Cifar100
from model.resnet_cifar import resnet32
from model.manager import Manager
from model.lwf import LWF
from model.icarl import Exemplars
from model.icarl import iCaRL
from losses.losses import AsymLoss, DKHLoss
from utils import plot

Cloning into 'incremental-learning-image-classification'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 1216 (delta 54), reused 47 (delta 16), pack-reused 1113
Receiving objects: 100% (1216/1216), 6.98 MiB | 5.28 MiB/s, done.
Resolving deltas: 100% (694/694), done.
renamed 'incremental-learning-image-classification/baselines.ipynb' -> './baselines.ipynb'
mv: cannot move 'incremental-learning-image-classification/data' to './data': Directory not empty
renamed 'incremental-learning-image-classification/distillation_targets.ipynb' -> './distillation_targets.ipynb'
mv: cannot move 'incremental-learning-image-classification/losses' to './losses': Directory not empty
mv: cannot move 'incremental-learning-image-classification/model' to './model': Directory not empty
renamed 'incremental-learning-image-classification/README.md' -> './README.md'
mv: cannot move 'incremental-learning-image-

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Arguments

In [5]:
# Directories
DATA_DIR = 'data'       # Directory where the dataset will be downloaded

# Settings
DEVICE = 'cuda'

# Dataset
RANDOM_STATES = [658, 423, 422]      # For reproducibility of results                        
                                     # Note: different random states give very different
                                     # splits and therefore very different results.

NUM_CLASSES = 100       # Total number of classes

VAL_SIZE = 0.1          # Proportion of validation set with respect to training set (between 0 and 1)

# Training
BATCH_SIZE = 64         # Batch size (iCaRL sets this to 128)
LR = 2                  # Initial learning rate
                       
MOMENTUM = 0.9          # Momentum for stochastic gradient descent (SGD)
WEIGHT_DECAY = 1e-5     # Weight decay from iCaRL

NUM_RUNS = 3            # Number of runs of every method
                        # Note: this should be at least 3 to have a fair benchmark

NUM_EPOCHS = 70         # Total number of training epochs
MILESTONES = [49, 63]   # Step down policy from iCaRL (MultiStepLR)
                        # Decrease the learning rate by gamma at each milestone
GAMMA = 0.2             # Gamma factor from iCaRL

## iCaRL

In [6]:
class iCaRLwithLoss(iCaRL):
    """Implement iCaRL, a strategy for simultaneously learning classifiers and a
    feature representation in the class-incremental setting.
    """

    def __init__(self, device, net, lr, criterion, momentum, weight_decay, milestones, gamma, num_epochs, batch_size, train_transform, test_transform):
        
        super().__init__(device, net, lr, momentum, weight_decay, milestones, gamma, num_epochs, batch_size, train_transform, test_transform)
        # Loss function
        self.criterion = criterion


### Data preparation

In [7]:
# Transformations for Learning Without Forgetting
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

In [8]:
train_subsets = [[] for i in range(NUM_RUNS)]
val_subsets = [[] for i in range(NUM_RUNS)]
test_subsets = [[] for i in range(NUM_RUNS)]

for run_i in range(NUM_RUNS):
    for split_i in range(10):
        if run_i+split_i == 0: # Download dataset only at first instantiation
            download = True
        else:
            download = False

        # Create CIFAR100 dataset
        train_dataset = Cifar100(DATA_DIR, train=True, download=download, random_state=RANDOM_STATES[run_i], transform=train_transform)
        test_dataset = Cifar100(DATA_DIR, train=False, download=False, random_state=RANDOM_STATES[run_i], transform=test_transform)
    
        # Subspace of CIFAR100 of 10 classes
        train_dataset.set_classes_batch(train_dataset.batch_splits[split_i]) 
        test_dataset.set_classes_batch([test_dataset.batch_splits[i] for i in range(0, split_i+1)])

        # Define train and validation indices
        train_indices, val_indices = train_dataset.train_val_split(VAL_SIZE, RANDOM_STATES[run_i])

        # Define subsets
        train_subsets[run_i].append(Subset(train_dataset, train_indices))
        val_subsets[run_i].append(Subset(train_dataset, val_indices))
        test_subsets[run_i].append(test_dataset)

Extracting data/cifar-100-python.tar.gz to data


### Execution

In [9]:
# iCaRL hyperparameters
LR = 2
MOMENTUM = 0.9
WEIGHT_DECAY = 0.00001
MILESTONES = [49, 63]
GAMMA = 0.2
NUM_EPOCHS = 70
BATCH_SIZE = 64

In [ ]:
# Define what tests to run
TEST_ICARL = True # Run test with iCaRL (exemplars + train dataset)
TEST_HYBRID1 = False # Run test with hybrid1

# Initialize logs
logs_icarl = [[] for _ in range(NUM_RUNS)]
logs_hybrid1 = [[] for _ in range(NUM_RUNS)]

for run_i in range(NUM_RUNS-2):
    net = resnet32()
    criterion = AsymLoss(nn.BCEWithLogitsLoss)
    icarl = iCaRLwithLoss(DEVICE, net, LR, criterion, MOMENTUM, WEIGHT_DECAY, MILESTONES, GAMMA, NUM_EPOCHS, BATCH_SIZE, train_transform, test_transform)

    for split_i in range(1):
        print(f"## Split {split_i} of run {run_i} ##")
        
        train_logs = icarl.incremental_train(split_i, train_subsets[run_i][split_i], val_subsets[run_i][split_i])

        if TEST_ICARL:
            logs_icarl[run_i].append({})

            acc, all_targets, all_preds = icarl.test(test_subsets[run_i][split_i], train_subsets[run_i][split_i])

            logs_icarl[run_i][split_i]['accuracy'] = acc
            logs_icarl[run_i][split_i]['conf_mat'] = confusion_matrix(all_targets.to('cpu'), all_preds.to('cpu'))

            logs_icarl[run_i][split_i]['train_loss'] = train_logs[0]
            logs_icarl[run_i][split_i]['train_accuracy'] = train_logs[1]
            logs_icarl[run_i][split_i]['val_loss'] = train_logs[2]
            logs_icarl[run_i][split_i]['val_accuracy'] = train_logs[3]

        if TEST_HYBRID1:
            logs_hybrid1[run_i].append({})

            acc, all_targets, all_preds = icarl.test_without_classifier(test_subsets[run_i][split_i])

            logs_hybrid1[run_i][split_i]['accuracy'] = acc
            logs_hybrid1[run_i][split_i]['conf_mat'] = confusion_matrix(all_targets.to('cpu'), all_preds.to('cpu'))

## Split 0 of run 0 ##
Length of exemplars set: 0
Epoch: 1, LR: [2]
Train loss: 0.20704161716359, Train accuracy: 0.14866071428571428
Validation loss: 0.17530382105282374, Validation accuracy: 0.25223214285714285
Epoch: 2, LR: [2]
Train loss: 0.18046391308307647, Train accuracy: 0.22254464285714284
Validation loss: 0.20530323897089278, Validation accuracy: 0.16964285714285715
Epoch: 3, LR: [2]


### Plots

In [ ]:
train_loss = [[logs_icarl[run_i][i]['train_loss'] for i in range(10)] for run_i in range(NUM_RUNS)]
train_accuracy = [[logs_icarl[run_i][i]['train_accuracy'] for i in range(10)] for run_i in range(NUM_RUNS)]
val_loss = [[logs_icarl[run_i][i]['val_loss'] for i in range(10)] for run_i in range(NUM_RUNS)]
val_accuracy = [[logs_icarl[run_i][i]['val_accuracy'] for i in range(10)] for run_i in range(NUM_RUNS)]
test_accuracy = [[logs_icarl[run_i][i]['accuracy'] for i in range(10)] for run_i in range(NUM_RUNS)]

train_loss = np.array(train_loss)
train_accuracy = np.array(train_accuracy)
val_loss = np.array(val_loss)
val_accuracy = np.array(val_accuracy)
test_accuracy = np.array(test_accuracy)

train_loss_stats = np.array([train_loss.mean(0), train_loss.std(0)]).transpose()
train_accuracy_stats = np.array([train_accuracy.mean(0), train_accuracy.std(0)]).transpose()
val_loss_stats = np.array([val_loss.mean(0), val_loss.std(0)]).transpose()
val_accuracy_stats = np.array([val_accuracy.mean(0), val_accuracy.std(0)]).transpose()
test_accuracy_stats = np.array([test_accuracy.mean(0), test_accuracy.std(0)]).transpose()

In [ ]:
plot.train_val_scores(train_loss_stats, train_accuracy_stats, val_loss_stats, val_accuracy_stats)

In [ ]:
plot.test_scores(test_accuracy_stats)